In [12]:
#For use in Google drive
from google.colab import drive
drive.mount('/content/drive')
tweetPath = "/content/drive/MyDrive/Codes/Twitter Data"


ModuleNotFoundError: No module named 'google'

In [13]:
#For running locally
tweetPath = "Twitter Data"
stockPath = "Stock_Max"

In [56]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from nltk.sentiment import SentimentIntensityAnalyzer
from functools import reduce
from IPython.display import display
import nltk
nltk.download('vader_lexicon')
from tqdm import tqdm
tqdm.pandas()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\liche\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 1000)

In [71]:
tweets = pd.read_csv('processed_dataframe.csv',index_col=0)
tweets_exclRT = tweets[~tweets['post'].str.contains("RT").astype(bool)].reset_index(drop=True)

prices = reduce(lambda x,y:x.merge(y,how="outer", left_index=True,right_index=True),\
[pd.read_csv(os.path.join(stockPath,filename)).loc[:,["Date","Adj Close"]].set_index('Date')\
.set_axis([filename[:-4]], axis=1) for filename in os.listdir(stockPath)])
long_prices = pd.melt(prices,ignore_index=False,var_name="ticker",value_name="adj. close")

,date,post,retweets,favorites,ticker,sentiment,adj. close
0,2021-10-05,RT @WSBConsensus: Apple ( $AAPL ) suppliers wa...,1.0,0.0,AAPL,-0.3818,140.904358
1,2021-10-05,$AAPL suppliers warned that energy crunch in C...,0.0,1.0,AAPL,-0.3818,140.904358
2,2021-10-05,Apple ( $AAPL ) suppliers warned that energy c...,1.0,9.0,AAPL,-0.3818,140.904358
3,2021-10-05,RT @ElkanTrades: Trading just like life someti...,3.0,0.0,AAPL,0.3612,140.904358
4,2021-10-05,"SPDR-ETF Report For Tuesday, Oct. 5 $FXE $GSX ...",0.0,0.0,AAPL,0.0000,140.904358
...,...,...,...,...,...,...,...
605933,2021-11-10,Top Trending Stocks on #fintwit: \n1. $COIN 🥇\...,0.0,2.0,TSLA,0.2023,1067.949951
605934,2021-11-10,RT @Keubiko: $TSLA $1300 weekly call buyer aft...,9.0,0.0,TSLA,0.0000,1067.949951
605935,2021-11-10,I‘m sure that his timing was pure coincidence....,1.0,20.0,TSLA,0.3182,1067.949951
605936,2021-11-10,RT @garyblack00: Not sure why we’re still talk...,58.0,0.0,TSLA,-0.6769,1067.949951


In [81]:

tweets.index

,date,post,retweets,favorites,ticker,sentiment
0,2021-10-05,RT @WSBConsensus: Apple ( $AAPL ) suppliers wa...,1.0,0.0,AAPL,-0.3818
1,2021-10-05,$AAPL suppliers warned that energy crunch in C...,0.0,1.0,AAPL,-0.3818
2,2021-10-05,Apple ( $AAPL ) suppliers warned that energy c...,1.0,9.0,AAPL,-0.3818
3,2021-10-05,RT @ElkanTrades: Trading just like life someti...,3.0,0.0,AAPL,0.3612
4,2021-10-05,"SPDR-ETF Report For Tuesday, Oct. 5 $FXE $GSX ...",0.0,0.0,AAPL,0.0000
...,...,...,...,...,...,...
686521,2021-11-10,Top Trending Stocks on #fintwit: \n1. $COIN 🥇\...,0.0,2.0,TSLA,0.2023
686522,2021-11-10,RT @Keubiko: $TSLA $1300 weekly call buyer aft...,9.0,0.0,TSLA,0.0000
686523,2021-11-10,I‘m sure that his timing was pure coincidence....,1.0,20.0,TSLA,0.3182
686524,2021-11-10,RT @garyblack00: Not sure why we’re still talk...,58.0,0.0,TSLA,-0.6769


In [83]:
#Shouldn't be ran unless there are changes to twitter data source
#Use tweets = pd.read_csv('processed.csv') instead
tweets = pd.concat([pd.read_csv(os.path.join(tweetPath, filename),index_col=0)
.assign(ticker=filename[:-4].split("_")[2],date=filename[:-4].split("_")[3])\
for filename in os.listdir(tweetPath) if filename.endswith(".csv")])

tweets = tweets[tweets['post'].apply(lambda x:isinstance(x, str))]\
.drop(tweets[tweets['date'].apply(len)<2].index).reset_index(drop=True)

sia = SentimentIntensityAnalyzer()
tweets['sentiment'] = tweets['post'].progress_apply(lambda x:sia.polarity_scores(x)['compound'])
tweets['datetime'] = tweets['datetime'].apply(lambda x:x[:10])
tweets.rename({'datetime':'date'},inplace=True,axis=1)
tweets.to_csv('processed_dataframe.csv')

100%|██████████| 686526/686526 [03:42<00:00, 3085.56it/s]


PermissionError: [Errno 13] Permission denied: 'processed_dataframe.csv'